# ERP Visualization 
## Topo Plots & Waveforms
### N170 (2 ROIs)

---
Copyright 2024 [Aaron J Newman](https://github.com/aaronjnewman), [NeuroCognitive Imaging Lab](http://ncil.science), [Dalhousie University](https://dal.ca)

Released under the [The 3-Clause BSD License](https://opensource.org/licenses/BSD-3-Clause)

---

# Initialization

In [ ]:
component = 'n170'

## Load in the necessary libraries/packages we'll need

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op
from glob import glob
from pathlib import Path
import json
import mne
mne.set_log_level('error')

## Read Parameters from config.yml

Will import study-level parameters from `config.yml` in `bids_root`

In [ ]:
# this shouldn't change if you run this script from its default location in code/import
bids_root = '../..'

config_file = op.join(bids_root, 'config.json')
config = json.load(open(config_file))

study_name = config['Study']['Name']
study_name = config['Study']['TaskName']
data_type = config['data_type']
eog = {k: v for d in config['eog'] for k, v in d.items()}
montage_fname = config['montage_fname']
event_id = {k: v.pop() for d in config['events'] for k, v in d.items()}

n_jobs = config['Preprocessing']['n_jobs']

cfg = config['components'][component] #{k: v for d in config['analysis_settings'] for k, v in d.items()}

## Time windows of interest

In [ ]:
# Define the total length of the epoch 
# this can be less than what is in the input files; will use the crop function
tmin = cfg['t_min']
tmax =  cfg['t_max']
baseline = eval(cfg['baseline'])

# value obtained from butterfly plots visual inspection and added to config.yml
peak_lat = cfg['peak_lat']

tw_width = cfg['tw_width']

# Amount of time to shift event codes by, based on empirical testing with photocell
#  to determine lag between event code and actual stimulus appearance on screen.
#  For reasons that are unclear, we need to double this value (in sec) to get the correct shift
tshift = config['t_shift']

## Conditions and Contrasts of Interest

In [ ]:
conditions = list(event_id.values())

contrasts = {'CS-FF':['ConsonantString', 'FalseFont'],
             'RW-FF':['RealWord', 'FalseFont'],
             'PW-FF':['PseudoWord', 'FalseFont'],
             'NW-FF':['NovelWord', 'FalseFont'],
             'RW-CS':['RealWord', 'ConsonantString'],
             'PW-CS':['PseudoWord', 'ConsonantString'],
             'NW-CS':['NovelWord', 'ConsonantString'],
             'RW-PW':['RealWord', 'PseudoWord'],
             'RW-NW':['RealWord', 'NovelWord'],
             'NW-PW':['NovelWord', 'PseudoWord']
             }

## Paths

In [ ]:
source_path = op.join(bids_root, 'derivatives', 'erp_preprocessing')

derivatives_path = op.join(bids_root, 'derivatives', 'erp_visualization', component)
if Path(derivatives_path).exists() == False:
    Path(derivatives_path).mkdir(parents=True)

fig_path = op.join(derivatives_path, 'figures')
if Path(fig_path).exists() == False:
    Path(fig_path).mkdir(parents=True) 
    
epochs_suffix = '-epo.fif'
waveplot_stem = fig_path + '/waveforms_'
topoplot_stem = fig_path + '/topoplots_'

## Figure settings

In [ ]:
colors = {'FalseFont':sns.color_palette('colorblind')[0], 
          'ConsonantString':sns.color_palette('colorblind')[1], 
          'PseudoWord':sns.color_palette('colorblind')[2], 
          'NovelWord':sns.color_palette('colorblind')[3], 
          'RealWord':sns.color_palette('colorblind')[4]}

contr_colors = {'CS-FF':colors['ConsonantString'],
                'RW-FF':colors['RealWord'],
                'PW-FF':colors['PseudoWord'],
                'NW-FF':colors['NovelWord'],
                'RW-CS':colors['RealWord'],
                'PW-CS':colors['PseudoWord'],
                'NW-CS':colors['NovelWord'],
                'RW-PW':colors['RealWord'],
                'RW-NW':colors['RealWord'],
                'NW-PW':colors['NovelWord']
                }

linestyles = {'FalseFont':'-', 
              'ConsonantString': ':', 
              'PseudoWord':'-.', 
              'NovelWord':'--', 
              'RealWord':'-'}

# For big arrays of waveplots
waveplot_figsize = (18, 6)
fig_format = 'pdf'

## Define ROIs
clusters of electrodes to average over for waveform plots

In [ ]:
# convoluted unpacking from yaml
rois = {k: v for d in config['rois'] for k, v in d.items()}
rois = {roi:rois[roi]  for roi in cfg['rois']}

for roi, chs in rois.items():
    rois[roi]= [c.split(', ') for c in chs][0]

In [ ]:
# only works for single ROI
roi_chans = rois[cfg['rois'][0]]

## Subject list

In [ ]:
prefix = 'sub-'
subjects = sorted([s[-7:] for s in glob(source_path + '/' + prefix + '*')])

---
# Read in the data

When we read the data, we also crop the epochs as specified above, and time-shift the event onsets to match true stimulus timing

In [ ]:
epochs = {}
for subject in subjects:
    subj_path = op.join(source_path, subject, 'eeg')
    epochs[subject] = mne.read_epochs(str(subj_path + '/' + subject + '_task-' + task + '-epo.fif'),
                                         verbose=None, 
                                         preload=True)
    # correct for stimulus presentation delay
    epochs[subject]._raw_times = epochs[subject]._raw_times - tshift
    epochs[subject]._times_readonly = epochs[subject]._times_readonly - tshift
    
    epochs[subject].crop(tmin=tmin, tmax=tmax).apply_baseline(baseline)
    
    epochs[subject].set_montage(montage_fname)

In [ ]:
# epochs['sub-002']['FalseFont'].average().plot_joint()
# plt.show()

In [ ]:
# epochs['sub-002']['FalseFont'].drop_bad(reject={'eeg':400e-06}).average().plot_joint()
# plt.show()

## Create Evokeds

In [ ]:
evoked = {}
for cond in conditions:
    evoked[cond] = [epochs[subject][cond].average() for subject in subjects]

## Grand Averages

In [ ]:
gavg = {}
for cond in conditions:
    gavg[cond] = mne.grand_average(evoked[cond])

## Compute Contrasts
Differences between pairs of conditions

In [ ]:
evoked_diff = {}

for contr, conds in contrasts.items():
    evoked_diff[contr] = [mne.combine_evoked([ c1, c2],
                                                weights=[1, -1])
                             for (c1, c2) in zip(evoked[conds[0]], evoked[conds[1]])
                            ]

## Create mask identifying ROI electrodes

In [ ]:
chs = pd.Series(gavg[conditions[0]].ch_names)

roi_elec = [i for c in rois.values() for i in c ]
mask = chs.isin(roi_elec).to_numpy()
num_tp = gavg[conditions[0]].data.shape[1]
mask = np.repeat(mask[:, np.newaxis], num_tp, axis=1)

---
# Visualization

## Topo map time series for each condition

In [ ]:
times = np.arange(0., tmax, .050)

uv_range = 15

for cond in conditions:
    gavg[cond].plot_topomap(times, average=tw_width,
                            ch_type='eeg', 
                            show_names=False, sensors=False, contours=False, 
                            colorbar=True, 
                            vmin=-uv_range, vmax=uv_range,
                            title=(cond),
                            mask=mask,
                            mask_params=dict(marker='o', 
                                             markerfacecolor='w', 
                                             markeredgecolor='k',
                                             linewidth=0, 
                                             markersize=6)
                           )

### Average over N170 peak window

In [ ]:
uv_range = 10
for cond in conditions:
    gavg[cond].plot_topomap(peak_lat, average=tw_width,
                           ch_type='eeg', 
                           show_names=False, sensors=False, contours=False, 
                           colorbar=True, size=3, 
                           vmin=-uv_range, vmax=uv_range,
                           title=(cond),
                           mask=mask, 
                           mask_params=dict(marker='o', 
                                            markerfacecolor='w', 
                                            markeredgecolor='k',
                                            linewidth=0, 
                                            markersize=3)
                    ).savefig(topoplot_stem + 'IndivCond_' + cond + '.' + fig_format); 

## Waveforms

In [ ]:
ylim = {'eeg':[-10.5, 10]}

fig, axs = plt.subplots(1, 2, figsize=waveplot_figsize)    
ax = 0
for roi, chans in rois.items():
    if ax == 0:
        show=False
    else:
        show=True

    mne.viz.plot_compare_evokeds({c:evoked[c] for c in conditions},
                                 picks=chans, combine='mean',
                                 title=(roi),
                                 colors=colors, linestyles=linestyles,
                                 ylim=ylim,
                                 show_sensors='upper right', legend='lower right', 
                                 ci=False,
                                 axes=axs[ax], show=show
                                );
    # axs[ax].vlines([peak_lat-tw_width, peak_lat+tw_width], 
    #             ylim['eeg'][0], ylim['eeg'][1],
    #             color='gray', linestyle=':'
    #             )

    ax += 1

for ax in axs:
    # these don't show in the notebook, but do in the saved files
    ax.vlines([peak_lat-tw_width, peak_lat+tw_width], 
                    ylim['eeg'][0], ylim['eeg'][1],
                    color='gray', linestyle=':'
                    )
    # shade N170 time window
    ax.axvspan(peak_lat-tw_width, peak_lat+tw_width, alpha=0.5, color='lightgrey')

        
# Save images to files
fig.savefig(waveplot_stem + 'allcond_legend_tw.' + fig_format)

plt.show()

### GFP

In [ ]:
ylim = {'eeg':[0, 12]}

fig, axs = plt.subplots(1, 2, figsize=waveplot_figsize)    
ax = 0
for roi, chans in rois.items():
    if ax == 0:
        show=False
    else:
        show=True

    mne.viz.plot_compare_evokeds({c:evoked[c] for c in conditions},
                                 picks=chans, #combine='mean',
                                 title=(roi),
                                 colors=colors, linestyles=linestyles,
                                 ylim=ylim,
                                 show_sensors='upper right', legend='lower right', 
                                 ci=False,
                                 axes=axs[ax], show=show
                                );
    ax += 1

# Save images to files
fig.savefig(waveplot_stem + 'allcond_gfp_legend.' + fig_format)

plt.show()

### With CIs

In [ ]:
ylim = {'eeg':[-12, 10]}

fig, axs = plt.subplots(1, 2, figsize=waveplot_figsize)    
ax = 0
for roi, chans in rois.items():
    if ax == 0:
        show=False
    else:
        show=True

    mne.viz.plot_compare_evokeds({c:evoked[c] for c in conditions},
                                 picks=chans, combine='mean',
                                 title=(roi),
                                 colors=colors, linestyles=linestyles,
                                 ylim=ylim,
                                 show_sensors='upper right', legend='lower right', 
                                 ci=True,
                                 axes=axs[ax], show=show
                                );
    ax += 1

# Save images to files
fig.savefig(waveplot_stem + 'allcond_CIs.' + fig_format)

plt.show()

## Pairwise contrasts

In [ ]:
ylim = {'eeg':[-10.5, 10]}

for contr in contrasts:    
    fig, axs = plt.subplots(1, 2, figsize=waveplot_figsize)    
    ax = 0
    for roi, chans in rois.items():
        if ax == 0:
            show=False
        else:
            show=True

        mne.viz.plot_compare_evokeds({c:evoked[c] for c in contrasts[contr]},
                                     picks=chans, combine='mean',
                                     title=(contr + ' ' + roi),
                                     colors=[colors[i] for i in contrasts[contr]], linestyles=['-','--'],
                                     ylim=ylim,
                                 show_sensors='upper right', legend='lower right', 
                                     ci=False,
                                     axes=axs[ax], show=show
                                    );
        # Save images to files
        fig.savefig(waveplot_stem + contr + '.' + fig_format)
        
        ax += 1


plt.show()

---
# Difference Waves (Contrasts) 

In [ ]:

ylim = {'eeg':[-6, 3]}
component_timewin = (peak_lat - tw_width,
                     peak_lat + tw_width
                     )

for contr in contrasts:
    fig, axs = plt.subplots(1, 2, figsize=waveplot_figsize)   
    ax = 0
    for roi, chans in rois.items():
        if ax == 0:
            show=False
        else:
            show=True
            
        mne.viz.plot_compare_evokeds({contr:evoked_diff[contr]},
                                     picks=chans, combine='mean',
                                     title=(contr + ' ' + roi),
                                     ylim=ylim,
                                    #  vlines=[0, 
                                    #          component_timewin[0], 
                                    #          component_timewin[1]
                                    #          ],
                                     show_sensors='upper right', legend=False, 
                                     ci=True,
                                     colors=[contr_colors[contr]], 
                                     axes=axs[ax], show=show
                                    );
        ax += 1
        
    for ax in axs:
        # these don't show in the notebook, but do in the saved files
        ax.vlines([peak_lat-tw_width, peak_lat+tw_width], 
                       ylim['eeg'][0], ylim['eeg'][1],
                       color='gray', linestyle=':'
                      )

    # Save images to files
    fig.savefig(waveplot_stem + 'diff_' + contr + '.' + fig_format)
       

    plt.show()

## Topoplots

In [ ]:
times = np.arange(-.090, tmax - .050, .050)

uv_range = 4
contr_ff = ['CS-FF', 'RW-FF', 'PW-FF', 'NW-FF']
for contr in contr_ff:
     mne.grand_average(evoked_diff[contr]).plot_topomap(times, average=tw_width,
                                                       ch_type='eeg', 
                                                       show_names=False, sensors=False, contours=False, 
                                                       colorbar=True, 
                                                       vmin=-uv_range, vmax=uv_range,
                                                       mask=mask, 
                                                       title=(contr)
                                                      ).savefig(topoplot_stem + 'ts_' + contr + '.' + fig_format); 

### Plot with smaller scale for non-FF contrasts

In [ ]:
times = np.arange(-.090, tmax - .050, .050)

uv_range = 2

contr_no_ff = ['RW-CS', 'PW-CS', 'NW-CS', 'RW-PW', 'RW-NW', 'NW-PW']

for contr in contr_no_ff:
     mne.grand_average(evoked_diff[contr]).plot_topomap(times, average=tw_width,
                                                       ch_type='eeg', 
                                                       show_names=False, sensors=False, contours=False, 
                                                       colorbar=True, 
                                                       vmin=-uv_range, vmax=uv_range,
                                                       mask=mask, 
                                                       title=(contr)
                                                      ).savefig(topoplot_stem + 'ts_' + contr + '.' + fig_format); 

### Average over N170 peak window

In [ ]:
uv_range = 3

for contr in contr_ff:
    mne.grand_average(evoked_diff[contr]).plot_topomap(peak_lat, average=tw_width,
                                                       ch_type='eeg', 
                                                       show_names=False, sensors=False, contours=False, 
                                                       colorbar=True, size=3, 
                                                       vmin=-uv_range, vmax=uv_range,
                                                       title=(contr),
                                                       mask=mask, 
                                                       mask_params=dict(marker='o', 
                                                                        markerfacecolor='w', 
                                                                        markeredgecolor='k',
                                                                        linewidth=0, 
                                                                        markersize=3)
                                                ).savefig(topoplot_stem + contr + '.' + fig_format); 

In [ ]:
uv_range = 2

for contr in contr_no_ff:
    mne.grand_average(evoked_diff[contr]).plot_topomap(peak_lat, average=tw_width,
                                                       ch_type='eeg', 
                                                       show_names=False, sensors=False, contours=False, 
                                                       colorbar=True, size=3, 
                                                       vmin=-uv_range, vmax=uv_range,
                                                       title=(contr),
                                                       mask=mask, 
                                                       mask_params=dict(marker='o', 
                                                                        markerfacecolor='w', 
                                                                        markeredgecolor='k',
                                                                        linewidth=0, 
                                                                        markersize=3)
                                                ).savefig(topoplot_stem + contr + '.' + fig_format); 